In [46]:
import pandas as pd
import plotly.express as px
import re
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed

from DatabaseLocator import DatabaseLocator

In [68]:
Locator = DatabaseLocator()
netzero1 = pd.read_csv(filepath_or_buffer=Locator.get_onedrive_path("data/netzero-data/All-Subsystems-minute.csv"))

In [ ]:
netzero2 = pd.read_csv(filepath_or_buffer=Locator.get_onedrive_path("data/netzero-data/All-Subsystems-minute (1).csv"))

In [ ]:
metadata = pd.read_csv(filepath_or_buffer=Locator.get_onedrive_path("data/netzero-data/Metadata-minute.csv"))

In [127]:
netzero1["Timestamp"] = pd.to_datetime(netzero1["Timestamp"], format="%Y-%m-%d %H:%M:%S%z")

In [70]:
netzero1.loc[:50]

,Timestamp,TimeStamp_Count,DHW_ClothesWasherColdFlow,DHW_ClothesWasherHotFlow,DHW_DishwasherHotFlow,DHW_HeatPumpWaterHeaterEnergyTotal,DHW_HeatPumpWaterHeaterPowerTotal,DHW_ManifoldColdFlow,DHW_ManifoldHotFlow,DHW_MixValveColdFlow,...,Vent_HRVTempReturnIn,Vent_HRVTempSupplyOut,Vent_HRVfreshairpressuredifferential,Vent_HRVstaleairpressuredifferential,HVAC_HeatPumpIndoorUnitPower,HVAC_HeatPumpOutdoorUnitPower,HVAC_DehumidifierAirflow,HVAC_DehumidifierExitAirTemp,HVAC_DehumidifierInletAirTemp,HVAC_DehumidifierPower
0,2013-07-01 00:01:14-04:00,1,0.0,0.0,0.0,0.000,6.92,0.0,0.0,0.0,...,22.552117,23.758415,NaN,NaN,125.4000,922.6568,NaN,NaN,NaN,NaN
1,2013-07-01 00:02:14-04:00,2,0.0,0.0,0.0,0.132,6.92,0.0,0.0,0.0,...,22.523656,23.737489,NaN,NaN,127.0400,919.2424,NaN,NaN,NaN,NaN
2,2013-07-01 00:03:14-04:00,3,0.0,0.0,0.0,0.264,6.92,0.0,0.0,0.0,...,22.539345,23.742217,NaN,NaN,153.3050,920.8184,NaN,NaN,NaN,NaN
3,2013-07-01 00:04:14-04:00,4,0.0,0.0,0.0,0.396,6.92,0.0,0.0,0.0,...,22.539554,23.727526,NaN,NaN,143.4575,922.3944,NaN,NaN,NaN,NaN
4,2013-07-01 00:05:14-04:00,5,0.0,0.0,0.0,0.528,6.92,0.0,0.0,0.0,...,22.506425,23.705258,NaN,NaN,141.8150,925.2832,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,2013-07-01 01:37:17-04:00,97,0.0,0.0,0.0,12.672,6.92,0.0,0.0,0.0,...,22.250062,23.352873,NaN,NaN,163.9750,945.7696,NaN,NaN,NaN,NaN
97,2013-07-01 01:38:17-04:00,98,0.0,0.0,0.0,12.804,6.92,0.0,0.0,0.0,...,22.237499,23.342462,NaN,NaN,180.3900,958.3768,NaN,NaN,NaN,NaN
98,2013-07-01 01:39:17-04:00,99,0.0,0.0,0.0,12.936,6.92,0.0,0.0,0.0,...,22.274041,23.290242,NaN,NaN,99.1350,41.7536,NaN,NaN,NaN,NaN
99,2013-07-01 01:40:17-04:00,100,0.0,0.0,0.0,13.068,6.92,0.0,0.0,0.0,...,22.224018,23.278600,NaN,NaN,1.4650,40.4408,NaN,NaN,NaN,NaN


In [ ]:
netzero2.loc[:50]

In [ ]:
netzero1.describe().T

In [ ]:
netzero2.columns.difference(netzero1.columns)

In [ ]:
netzero1.columns[3]

In [ ]:
netzero1.columns.tolist()

In [ ]:
totals = [x for x in netzero1.columns if "Total" in x]
totals

In [ ]:
fig = px.scatter(netzero1, x="Timestamp", y="Load_KPlugLoadsPowerUsage", color="Load_StatusPlugLoadToaster")
fig.show()

In [62]:
plugs = metadata.loc[(metadata['Parameter'] == "Status_OnOff") & (metadata["Measurement_Location"] == "Kitchen") & (
    metadata["Unnamed: 0"].str.contains("Plug"))]["Unnamed: 0"].tolist()  #

In [128]:
kitchenplugs = netzero1.loc[0:10000, ["Timestamp", "Load_KPlugLoadsPowerUsage"] + plugs].copy()
kitchenplugs["Status"] = kitchenplugs[plugs].sum(axis=1)

In [8]:
(kitchenplugs[plugs] < 0) | (kitchenplugs[plugs] > 1)

,Load_StatusPlugLoadBlender,Load_StatusPlugLoadToasterOven,Load_StatusPlugLoadSlowCooker,Load_StatusPlugLoadToaster,Load_StatusPlugLoadHandMixer,Load_StatusPlugLoadCanOpener,Load_StatusPlugLoadCoffeeMaker
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
9996,False,False,False,False,False,False,False
9997,False,False,False,False,False,False,False
9998,False,False,False,False,False,False,False
9999,False,False,False,False,False,False,False


In [129]:
appliance_dict = {}
for appliance in plugs:
    name = " ".join(re.findall('[A-Z][^A-Z]*', appliance.split("Load")[-1]))
    appliance_switch = kitchenplugs[(kitchenplugs[appliance] != kitchenplugs[appliance].shift(1))][
                           ["Timestamp", appliance]][1:]
    appliance_dict[name] = pd.DataFrame(
        {"Timestamp": [appliance_switch.loc[index, "Timestamp"] for index in appliance_switch.index],
         "Appliance": name,
         "Color": ["green" if appliance_switch.loc[index, appliance] == 1 else "red" for index in
                   appliance_switch.index]})

appliance_dict

{'Blender':                    Timestamp Appliance  Color
 0  2013-07-01 06:03:17-04:00   Blender  green
 1  2013-07-01 06:10:17-04:00   Blender    red
 2  2013-07-02 06:03:29-04:00   Blender  green
 3  2013-07-02 06:10:29-04:00   Blender    red
 4  2013-07-03 06:03:28-04:00   Blender  green
 5  2013-07-03 06:10:28-04:00   Blender    red
 6  2013-07-04 06:03:27-04:00   Blender  green
 7  2013-07-04 06:10:27-04:00   Blender    red
 8  2013-07-05 06:03:29-04:00   Blender  green
 9  2013-07-05 06:10:29-04:00   Blender    red
 10 2013-07-06 09:13:27-04:00   Blender  green
 11 2013-07-06 09:19:27-04:00   Blender    red
 12 2013-07-07 09:13:27-04:00   Blender  green
 13 2013-07-07 09:19:27-04:00   Blender    red,
 'Toaster Oven':                    Timestamp     Appliance  Color
 0  2013-07-01 06:13:17-04:00  Toaster Oven  green
 1  2013-07-01 06:23:17-04:00  Toaster Oven    red
 2  2013-07-02 06:13:29-04:00  Toaster Oven  green
 3  2013-07-02 06:23:29-04:00  Toaster Oven    red
 4  2013-07-

In [ ]:
kitchenplugs

In [144]:
fig = px.scatter(kitchenplugs, x=[time.timestamp() for time in kitchenplugs["Timestamp"]], y="Load_KPlugLoadsPowerUsage", color="Status")
for appliance, df in appliance_dict.items():
    for index in df.index:
        fig.add_vline(x=df.loc[index, "Timestamp"].timestamp(), line_width=3, line_dash="dash",
                      annotation_text=df.loc[index, "Appliance"], line_color=df.loc[index, "Color"])  #

fig.update_layout(
    xaxis_tickformat = "yyyy-mm-dd HH:MM:SS"
)

fig.show()

In [91]:
kitchenplugs[kitchenplugs["Load_StatusPlugLoadToasterOven"] == 1][["Timestamp", "Load_StatusPlugLoadToasterOven"]]

,Timestamp,Load_StatusPlugLoadToasterOven
372,2013-07-01 06:13:17-04:00,1.0
373,2013-07-01 06:14:17-04:00,1.0
374,2013-07-01 06:15:17-04:00,1.0
375,2013-07-01 06:16:17-04:00,1.0
376,2013-07-01 06:17:17-04:00,1.0
...,...,...
517757,2014-06-30 06:19:23-04:00,1.0
517758,2014-06-30 06:20:23-04:00,1.0
517759,2014-06-30 06:21:23-04:00,1.0
517760,2014-06-30 06:22:23-04:00,1.0


In [92]:
fig = px.scatter(kitchenplugs, x="Timestamp",
                 y=[kitchenplugs["Load_StatusPlugLoadToasterOven" == 1], "Load_StatusPlugLoadBlender"])
fig.show()

ValueError: All arguments should have the same length. The length of argument `wide_variable_0` is 3578, whereas the length of  previously-processed arguments ['Timestamp'] is 518792

In [ ]:
fig = px.strip(kitchenplugs, x="Timestamp", y=["Load_StatusPlugLoadToasterOven", "Load_StatusPlugLoadBlender"])
fig.show()